[← Control Systems as Dynamical Systems](../../../getting_started/theory_to_python/control_systems_as_dynamical_systems.rst)


# Example: Crazyflie Multi-Sensor Fusion


Suppose we want to stabilize a quadrotor in 3D space. We need accurate position and velocity estimates to maintain stable flight. However, our sensors have complementary strengths and weaknesses: motion capture is precise but slow, the barometer drifts with temperature, and the IMU is noisy but fast.

In this notebook, we extend our state estimation framework to handle **multiple asynchronous sensors** with different characteristics. The Crazyflie quadrotor fuses motion capture, barometer, and IMU data using a Kalman filter that optimally weights each sensor based on its noise properties. This demonstrates how sensor fusion can achieve better performance than any single sensor alone.

## System Overview

We model the quadrotor dynamics, implement sensor-specific noise characteristics, design the multi-sensor observer, and integrate everything into a complete feedback system. This notebook bridges theory and practice for aerial robot state estimation with heterogeneous sensors.


## Block Diagram

The Crazyflie system has multiple sensor streams feeding a central estimator:

![Crazyflie multi-sensor fusion system](../../../_static/tutorial/theory_to_python/crazyflie_multisensor_system.svg)

where **setpoint generator** produces reference positions (e.g., hover at [0, 0, 1]), **position controller** outputs velocity commands to track the reference, **Crazyflie** is our quadrotor plant with 3D dynamics, **motion capture** is a 3D position sensor (10 Hz, 5mm accuracy), **barometer** measures altitude (20 Hz, 10cm accuracy, subject to drift), **IMU** measures velocity from accelerometer integration (100 Hz, noisy), and **Kalman Filter** is the state observer that fuses all three sensors optimally.

:::{note}
Different sensors operate at different rates and measure different aspects of the state. The KF intelligently weights each sensor based on its noise characteristics. For pedagogical clarity, we'll run all sensors synchronously in this notebook.
:::


### Discrete-time Dynamical Systems

We cast each component as a discrete-time dynamical system:

![Crazyflie as composition of dynamical systems](../../../_static/tutorial/theory_to_python/crazyflie_composition_of_dynamical_systems.svg)

We discuss the derivation of each dynamical system block below.

:::{note}
Notice the **three sensor blocks** all feeding into a single observer. The KF combines their measurements by concatenating them into a single measurement vector. Unlike the TurtleBot's nonlinear dynamics, the Crazyflie uses a linear constant-velocity model, making the KF exact (no linearization needed).
:::


### Block 1: Setpoint Generator

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/crazyflie_setpoint_block.svg"
      style="max-width: 100%;">
</div>

For this tutorial, we'll command the drone to hover at a fixed position, then transition to new positions after some time.

**State**: Target position $s_k = [x_r, y_r, z_r]^T$

**Evolution**: Step function that switches between waypoints at specified times

**Output**: Reference position

$$
r_k = s_k
$$

For now, we'll define waypoints as a simple list. Later, we'll integrate this into a stateful system.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykal import DynamicalSystem
from pykal.data_change import corrupt, prepare  # For realistic sensor noise


# Create hover trajectory: positions in meters
hover_waypoints = [
    (0.0, 0.0, 1.0),   # Start: hover at 1m altitude
    (1.0, 1.0, 1.5),   # Move to (1, 1, 1.5)
    (0.0, 0.0, 1.0),   # Return to start
]


# Simple helper function to get waypoint by index
def get_hover_waypoint(waypoints, idx):
    """Get waypoint as numpy array."""
    x_r, y_r, z_r = waypoints[idx % len(waypoints)]
    return np.array([[x_r], [y_r], [z_r]])


# Test getting waypoints
print("Waypoint 0:", get_hover_waypoint(hover_waypoints, 0).flatten())
print("Waypoint 1:", get_hover_waypoint(hover_waypoints, 1).flatten())
print("Waypoint 2:", get_hover_waypoint(hover_waypoints, 2).flatten())


### Block 2: Position Controller

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/crazyflie_controller_block.svg"
      style="max-width: 100%;">
</div>

The position controller generates velocity commands $[v_x, v_y, v_z]$ to drive the drone toward the reference position.

**Inputs**: Reference position $r = [x_r, y_r, z_r]^T$, position estimate $\hat{p} = [\hat{x}, \hat{y}, \hat{z}]^T$

**Outputs**: Velocity command

$$
\vec{v}_{cmd} = K_p (\vec{r} - \hat{\vec{p}})
$$

where $K_p$ is the proportional gain.

This controller is **stateless** (no internal dynamics), so we implement it as a pure function:


In [ ]:
def position_controller(
    phat: np.ndarray,  # Estimated position [x, y, z]
    r: np.ndarray,      # Reference position [x_r, y_r, z_r]
    Kp: float = 1.0,
    max_vel: float = 0.5  # m/s
) -> np.ndarray:
    """
    Proportional position controller for Crazyflie.
    
    Returns
    -------
    v_cmd : np.ndarray
        Velocity command [vx, vy, vz], shape (3,1)
    """
    # Position error
    error = r - phat
    
    # Proportional control
    v_cmd = Kp * error
    
    # Saturate
    v_cmd = np.clip(v_cmd, -max_vel, max_vel)
    
    return v_cmd


# Test the controller
phat_test = np.array([[0.0], [0.0], [0.8]])
r_test = np.array([[1.0], [0.5], [1.0]])
v_cmd = position_controller(phat_test, r_test, Kp=0.8)
print(f"Velocity command: {v_cmd.flatten()}")


### Block 3: Crazyflie Quadrotor (Plant)

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/crazyflie_plant_block.svg"
      style="max-width: 100%;">
</div>

We model the Crazyflie with a simplified **constant-velocity dynamics** in 3D:

**State**:

$$
x_k = \begin{bmatrix} x \\ y \\ z \\ v_x \\ v_y \\ v_z \end{bmatrix}
$$

**Inputs**: Velocity commands $u_k = \begin{bmatrix} v_{x,cmd} \\ v_{y,cmd} \\ v_{z,cmd} \end{bmatrix}$

**Dynamics**:

$$
\begin{aligned}
\vec{p}_{k+1} &= \vec{p}_k + \vec{v}_k \Delta t \\
\vec{v}_{k+1} &= \vec{v}_{cmd} \quad \text{(assume fast inner-loop control)}
\end{aligned}
$$

This is a linear system, making the KF exact (no linearization needed).

In [3]:
def crazyflie_f(xk: np.ndarray, uk: np.ndarray, dt: float) -> np.ndarray:
    """
    Crazyflie 3D constant-velocity dynamics.
    
    Parameters
    ----------
    xk : np.ndarray
        State [x, y, z, vx, vy, vz], shape (6,1)
    uk : np.ndarray
        Velocity command [vx_cmd, vy_cmd, vz_cmd], shape (3,1)
    dt : float
        Timestep
    
    Returns
    -------
    xk_next : np.ndarray
        Next state, shape (6,1)
    """
    pos = xk[:3]
    vel = xk[3:]
    
    # Update position
    pos_new = pos + vel * dt
    
    # Update velocity (assume instantaneous response to command)
    vel_new = uk
    
    return np.vstack([pos_new, vel_new])


def crazyflie_h_mocap(xk: np.ndarray) -> np.ndarray:
    """Motion capture measurement: observe [x, y, z]."""
    return xk[:3]


def crazyflie_h_baro(xk: np.ndarray) -> np.ndarray:
    """Barometer measurement: observe z only."""
    return xk[2:3]


def crazyflie_h_imu(xk: np.ndarray) -> np.ndarray:
    """IMU measurement: observe velocity [vx, vy, vz] (from accelerometer integration)."""
    return xk[3:]


# Create plant DynamicalSystem
crazyflie_plant = DynamicalSystem(f=crazyflie_f, h=crazyflie_h_mocap, state_name="xk")

# Test
xk_test = np.array([[0.0], [0.0], [1.0], [0.0], [0.0], [0.0]])
uk_test = np.array([[0.1], [0.0], [0.05]])
dt = 0.01

xk_next, yk = crazyflie_plant.step(
    param_dict={"xk": xk_test, "uk": uk_test, "dt": dt},
    return_state=True
)
print("After one step:")
print(f"  State: {xk_next.flatten()}")
print(f"  Mocap measurement: {yk.flatten()}")

After one step:
  State: [0.   0.   1.   0.1  0.   0.05]
  Mocap measurement: [0. 0. 1.]


### Block 4: Multi-Sensor Measurements

We have three sensors, each measuring different aspects of the state:

#### Sensor 1: Motion Capture
- **Measures**: 3D position $[x, y, z]$
- **Rate**: 10 Hz (simulated)
- **Noise**: $\sigma = 5$ mm per axis
- **Characteristics**: High accuracy, low rate

#### Sensor 2: Barometer
- **Measures**: Altitude $z$ only
- **Rate**: 20 Hz
- **Noise**: $\sigma = 10$ cm
- **Characteristics**: Moderate accuracy, subject to pressure drift

#### Sensor 3: IMU (Accelerometer)
- **Measures**: Velocity $[v_x, v_y, v_z]$ (integrated acceleration)
- **Rate**: 100 Hz (simulated)
- **Noise**: $\sigma = 0.1$ m/s per axis
- **Characteristics**: High rate, accumulates drift

For this synchronous simulation, we'll concatenate all measurements:

$$
y_k = \begin{bmatrix} x \\ y \\ z \\ z \\ v_x \\ v_y \\ v_z \end{bmatrix} + v_k
$$

where $v_k \sim \mathcal{N}(0, R)$ with block-diagonal $R$:

$$
R = \text{diag}(R_{mocap}, R_{baro}, R_{imu})
$$


In [4]:
def generate_multisensor_measurement(
    xk: np.ndarray,
    R_mocap: np.ndarray,
    R_baro: np.ndarray,
    R_imu: np.ndarray,
    use_realistic_corruption: bool = True
) -> tuple:
    """
    Generate noisy measurements from all three sensors.
    
    Parameters
    ----------
    use_realistic_corruption : bool
        If True, uses data_change methods for sensor-specific corruption.
        If False, uses simple Gaussian noise.
    
    Returns
    -------
    yk_combined : np.ndarray
        Concatenated measurement vector [mocap(3), baro(1), imu(3)] = (7,1)
    R_combined : np.ndarray
        Block-diagonal noise covariance (7, 7)
    """
    # Clean measurements
    y_mocap = crazyflie_h_mocap(xk)  # [x, y, z]
    y_baro = crazyflie_h_baro(xk)     # [z]
    y_imu = crazyflie_h_imu(xk)       # [vx, vy, vz]
    
    if use_realistic_corruption:
        # Realistic sensor-specific corruption using data_change
        
        # Mocap: High precision, Gaussian noise only
        mocap_corrupted = y_mocap.flatten()
        mocap_corrupted = mocap_corrupted + np.random.multivariate_normal(
            np.zeros(3), R_mocap
        )
        y_mocap_noisy = mocap_corrupted.reshape(-1, 1)
        
        # Barometer: Drift + bias + Gaussian noise
        baro_corrupted = y_baro.item()
        baro_corrupted = corrupt.with_bias(baro_corrupted, bias=0.02)  # 2cm offset
        # Drift is cumulative, so we skip it in single-shot measurements
        baro_corrupted = baro_corrupted + np.random.normal(0, np.sqrt(R_baro[0,0]))
        y_baro_noisy = np.array([[baro_corrupted]])
        
        # IMU: Bias + Gaussian noise + occasional spikes (vibration)
        imu_corrupted = y_imu.flatten()
        imu_corrupted = imu_corrupted + np.random.multivariate_normal(
            np.zeros(3), R_imu
        )
        # Add vibration spikes with 3% probability
        if np.random.rand() < 0.03:
            spike_idx = np.random.randint(0, 3)
            imu_corrupted[spike_idx] += np.random.choice([-1, 1]) * 0.2
        y_imu_noisy = imu_corrupted.reshape(-1, 1)
        
    else:
        # Simple Gaussian noise (original method)
        y_mocap_noisy = y_mocap + np.random.multivariate_normal(
            np.zeros(3), R_mocap
        ).reshape(-1, 1)
        
        y_baro_noisy = y_baro + np.random.multivariate_normal(
            np.zeros(1), R_baro
        ).reshape(-1, 1)
        
        y_imu_noisy = y_imu + np.random.multivariate_normal(
            np.zeros(3), R_imu
        ).reshape(-1, 1)
    
    # Concatenate
    yk_combined = np.vstack([y_mocap_noisy, y_baro_noisy, y_imu_noisy])
    
    # Block-diagonal R
    R_combined = np.block([
        [R_mocap, np.zeros((3, 1)), np.zeros((3, 3))],
        [np.zeros((1, 3)), R_baro, np.zeros((1, 3))],
        [np.zeros((3, 3)), np.zeros((3, 1)), R_imu]
    ])
    
    return yk_combined, R_combined


# Define sensor noise covariances
R_mocap = np.diag([0.005, 0.005, 0.005])  # 5mm std
R_baro = np.array([[0.1]])                 # 10cm std
R_imu = np.diag([0.1, 0.1, 0.1])          # 0.1 m/s std

# Test measurement generation
xk_test = np.array([[0.5], [0.3], [1.0], [0.1], [0.05], [0.0]])
yk_combined, R_combined = generate_multisensor_measurement(
    xk_test, R_mocap, R_baro, R_imu, use_realistic_corruption=True
)
print(f"Combined measurement shape: {yk_combined.shape}")
print(f"R_combined shape: {R_combined.shape}")
print(f"Measurements: {yk_combined.flatten()}")

Combined measurement shape: (7, 1)
R_combined shape: (7, 7)
Measurements: [ 0.33493817  0.17683494  0.8861692   1.26591931 -0.60924457  0.17251324
  0.32637674]


### Block 5: Multi-Sensor Kalman Filter

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/crazyflie_observer_block.svg"
      style="max-width: 100%;">
</div>

The KF fuses all three sensors by stacking their measurement models.

**State transition Jacobian** (linear system, so F is constant):

$$
F = \begin{bmatrix}
I_3 & I_3 \Delta t \\
0 & I_3
\end{bmatrix}
$$

**Measurement Jacobian** (stacked for all sensors):

$$
H = \begin{bmatrix}
I_3 & 0 \\ 
\begin{bmatrix}0 & 0 & 1\end{bmatrix} & 0 \\
0 & I_3
\end{bmatrix}
$$

This $H$ matrix extracts:
- Rows 1-3: position for mocap
- Row 4: z-position for barometer
- Rows 5-7: velocity for IMU

**Process noise** $Q$: Small uncertainty in position, larger in velocity.

In [5]:
from pykal.algorithm_library.estimators.kf import KF


def compute_F_crazyflie(dt: float) -> np.ndarray:
    """Compute state transition Jacobian for Crazyflie (constant)."""
    I3 = np.eye(3)
    return np.block([
        [I3, I3 * dt],
        [np.zeros((3, 3)), I3]
    ])


def compute_H_multisensor() -> np.ndarray:
    """
    Compute measurement Jacobian for multi-sensor fusion.
    
    Measurement order: [mocap(3), baro(1), imu(3)] = 7 measurements
    State order: [pos(3), vel(3)] = 6 states
    """
    H = np.array([
        # Mocap: measures x, y, z (first 3 states)
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        # Barometer: measures z only (3rd state)
        [0, 0, 1, 0, 0, 0],
        # IMU: measures vx, vy, vz (last 3 states)
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]
    ])
    return H


def h_multisensor(xk: np.ndarray) -> np.ndarray:
    """
    Multi-sensor measurement function (clean, no noise).
    
    Concatenates all sensor measurements: [mocap(3), baro(1), imu(3)] = 7D
    """
    y_mocap = crazyflie_h_mocap(xk)  # [x, y, z]
    y_baro = crazyflie_h_baro(xk)     # [z]
    y_imu = crazyflie_h_imu(xk)       # [vx, vy, vz]
    return np.vstack([y_mocap, y_baro, y_imu])


# Process noise
Q_crazyflie = np.diag([0.001, 0.001, 0.001, 0.1, 0.1, 0.1])

# Create observer
crazyflie_observer = DynamicalSystem(
    f=KF.f,
    h=KF.h,
    state_name="xhat_P"
)

# Test observer with multi-sensor measurement
xhat_0 = np.array([[0.0], [0.0], [1.0], [0.0], [0.0], [0.0]])
P_0 = np.diag([0.1, 0.1, 0.1, 1.0, 1.0, 1.0])
xhat_P_test = (xhat_0, P_0)

yk_test, R_test = generate_multisensor_measurement(
    xhat_0, R_mocap, R_baro, R_imu
)

xhat_P_new, xhat_out = crazyflie_observer.step(
    param_dict={
        "xhat_P": xhat_P_test,
        "yk": yk_test,
        "f": crazyflie_f,
        "f_params": {"xk": xhat_0, "uk": np.zeros((3, 1)), "dt": 0.01},
        "h": h_multisensor,
        "h_params": {"xk": xhat_0},
        "Fk": compute_F_crazyflie(dt=0.01),
        "Qk": Q_crazyflie,
        "Hk": compute_H_multisensor(),
        "Rk": R_test
    },
    return_state=True
)

print("Multi-sensor KF estimate:")
print(xhat_out.flatten())

Multi-sensor KF estimate:
[-0.02613791 -0.16088758  0.96156147  0.13851319  0.03199376  0.10310147]


## Simulation

![Complete Crazyflie system](../../../_static/tutorial/theory_to_python/crazyflie_composition_of_dynamical_systems.svg)

We now simulate the complete closed-loop system, integrating all five dynamical components:
1. **Setpoint Generator** → reference position $r_k$
2. **Position Controller** → velocity commands $u_k$ (using $r_k$ and $\hat{p}_k$)
3. **Crazyflie Plant** → true state evolution
4. **Multi-Sensor Array** → measurements $y_k$ from mocap, baro, IMU
5. **Kalman Filter** → fused state estimate $\hat{x}_k$ (using $u_k$ and $y_k$)


### System Parameters


In [ ]:
# Time parameters
dt = 0.01  # Sampling time (seconds) - 100 Hz
switch_time = 20.0  # Time at each waypoint (seconds)

# Controller gains
Kp = 0.8  # Position control gain

# Kalman filter parameters
Q = np.diag([0.001, 0.001, 0.001, 0.1, 0.1, 0.1])  # Process noise
R_mocap = np.diag([0.005, 0.005, 0.005])  # Motion capture noise (5mm)
R_baro = np.array([[0.1]])  # Barometer noise (10cm)
R_imu = np.diag([0.1, 0.1, 0.1])  # IMU noise (0.1 m/s)


### Initial Conditions


In [ ]:
# Initial states
waypoint_idx = 0  # Start at first waypoint
time_at_waypoint = 0.0  # Time spent at current waypoint

xk = np.array([[0.0], [0.0], [1.0], [0.0], [0.0], [0.0]])  # Plant state: hover at 1m
xhat = np.array([[0.0], [0.0], [1.0], [0.0], [0.0], [0.0]])  # Observer estimate
P = np.diag([0.1, 0.1, 0.1, 1.0, 1.0, 1.0])  # Covariance matrix
xhat_P = (xhat, P)  # Observer state tuple

# Storage for plotting
time_hist = []
reference_hist = []
true_state_hist = []
estimate_hist = []
measurement_mocap_hist = []
measurement_baro_hist = []
command_hist = []
error_hist = []


### Simulate


In [ ]:
# Simulation time
T_sim = 60.0  # seconds
time_steps = np.arange(0, T_sim, dt)

# Import KF from algorithm library
from pykal.algorithm_library.estimators import kf as KF_module

# Create observer DynamicalSystem
observer = DynamicalSystem(f=KF_module.f, h=KF_module.h, state_name="xhat_P")

# Run closed-loop simulation
for tk in time_steps:
    # 1. Setpoint generator (simple switching logic)
    time_at_waypoint += dt
    if time_at_waypoint >= switch_time:
        waypoint_idx = (waypoint_idx + 1) % len(hover_waypoints)
        time_at_waypoint = 0.0
    
    rk = get_hover_waypoint(hover_waypoints, waypoint_idx)
    
    # 2. Extract position estimate from observer
    xhat = observer.h(xhat_P)
    phat = xhat[:3]
    
    # 3. Position controller step
    uk = position_controller(phat, rk, Kp=Kp)
    
    # 4. Plant step (true dynamics)
    xk, _ = crazyflie_plant.step(
        return_state=True,
        param_dict={"xk": xk, "uk": uk, "dt": dt}
    )
    
    # 5. Generate multi-sensor measurements
    yk_combined, R_combined = generate_multisensor_measurement(
        xk, R_mocap, R_baro, R_imu, use_realistic_corruption=True
    )
    
    # 6. Observer step (KF)
    Fk = compute_F_crazyflie(dt)
    Hk = compute_H_multisensor()
    
    xhat_P, xhat_obs = observer.step(
        return_state=True,
        param_dict={
            "xhat_P": xhat_P,
            "yk": yk_combined,
            "f": crazyflie_f,
            "f_params": {"xk": xhat, "uk": uk, "dt": dt},
            "h": h_multisensor,
            "h_params": {"xk": xhat},
            "Fk": Fk,
            "Qk": Q,
            "Hk": Hk,
            "Rk": R_combined,
        },
    )
    
    # Store results
    time_hist.append(tk)
    reference_hist.append(rk.flatten())
    true_state_hist.append(xk.flatten())
    estimate_hist.append(xhat.flatten())
    measurement_mocap_hist.append(yk_combined[:3].flatten())
    measurement_baro_hist.append(yk_combined[3].item())
    command_hist.append(uk.flatten())
    error_hist.append((xk - xhat).flatten())

# Convert to numpy arrays
reference_hist = np.array(reference_hist)
true_state_hist = np.array(true_state_hist)
estimate_hist = np.array(estimate_hist)
measurement_mocap_hist = np.array(measurement_mocap_hist)
measurement_baro_hist = np.array(measurement_baro_hist)
command_hist = np.array(command_hist)
error_hist = np.array(error_hist)


### Visualize

We can visualize the pertinent values of our system to assure correct behavior. Note how the Kalman filter fuses three heterogeneous sensors to provide accurate state estimates despite each sensor's limitations.


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(15, 10))

# Plot 1: 3D Trajectory
ax = fig.add_subplot(2, 3, 1, projection='3d')
ax.plot(
    true_state_hist[:, 0],
    true_state_hist[:, 1],
    true_state_hist[:, 2],
    'b-',
    label='True',
    linewidth=2,
    alpha=0.7
)
ax.plot(
    estimate_hist[:, 0],
    estimate_hist[:, 1],
    estimate_hist[:, 2],
    'r--',
    label='Estimate',
    linewidth=2,
    alpha=0.7
)
ax.scatter(
    reference_hist[:, 0],
    reference_hist[:, 1],
    reference_hist[:, 2],
    c='green',
    s=100,
    marker='*',
    label='Setpoints'
)
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title('3D Trajectory', fontweight='bold')
ax.legend()

# Plot 2: XY Trajectory (top view)
ax = fig.add_subplot(2, 3, 2)
ax.plot(
    true_state_hist[:, 0],
    true_state_hist[:, 1],
    'b-',
    label='True',
    linewidth=2,
    alpha=0.7
)
ax.plot(
    estimate_hist[:, 0],
    estimate_hist[:, 1],
    'r--',
    label='Estimate',
    linewidth=2,
    alpha=0.7
)
ax.scatter(
    reference_hist[:, 0],
    reference_hist[:, 1],
    c='green',
    s=100,
    marker='*',
    label='Setpoints',
    zorder=5
)
ax.set_xlabel('X Position (m)', fontsize=11)
ax.set_ylabel('Y Position (m)', fontsize=11)
ax.set_title('XY Trajectory (Top View)', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')

# Plot 3: Z Position vs Time (with all sensors)
ax = fig.add_subplot(2, 3, 3)
ax.plot(time_hist, true_state_hist[:, 2], 'b-', label='True Z', linewidth=2, alpha=0.7)
ax.plot(time_hist, estimate_hist[:, 2], 'r--', label='Estimated Z', linewidth=2, alpha=0.7)
ax.scatter(
    time_hist[::100],
    measurement_mocap_hist[::100, 2],
    c='green',
    s=5,
    alpha=0.3,
    label='Mocap Z'
)
ax.scatter(
    time_hist[::50],
    measurement_baro_hist[::50],
    c='purple',
    s=5,
    alpha=0.3,
    label='Baro Z'
)
ax.set_ylabel('Z Position (m)', fontsize=11)
ax.set_title('Altitude: Multi-Sensor Fusion', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: X Position vs Time
ax = fig.add_subplot(2, 3, 4)
ax.plot(time_hist, true_state_hist[:, 0], 'b-', label='True X', alpha=0.7)
ax.plot(time_hist, estimate_hist[:, 0], 'r--', label='Estimated X', alpha=0.7)
ax.scatter(
    time_hist[::100],
    measurement_mocap_hist[::100, 0],
    c='gray',
    s=5,
    alpha=0.3,
    label='Mocap'
)
ax.set_ylabel('X Position (m)', fontsize=11)
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_title('X Position Over Time', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 5: Velocity Commands
ax = fig.add_subplot(2, 3, 5)
ax.plot(time_hist, command_hist[:, 0], label='Vx cmd', alpha=0.7)
ax.plot(time_hist, command_hist[:, 1], label='Vy cmd', alpha=0.7)
ax.plot(time_hist, command_hist[:, 2], label='Vz cmd', alpha=0.7)
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('Velocity Command (m/s)', fontsize=11)
ax.set_title('Control Commands', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 6: Position Estimation Error
ax = fig.add_subplot(2, 3, 6)
position_error = np.sqrt(
    error_hist[:, 0]**2 + error_hist[:, 1]**2 + error_hist[:, 2]**2
)
ax.plot(time_hist, position_error * 1000, 'm-', linewidth=1.5)  # Convert to mm
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('Position Error (mm)', fontsize=11)
ax.set_title('3D Position Estimation Error', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean position error: {np.mean(position_error)*1000:.2f} mm")
print(f"Max position error: {np.max(position_error)*1000:.2f} mm")


## Callback Wrapper

Just as we did with the cruise control and TurtleBot systems, we can wrap the **entire Crazyflie navigation system** in a single callback. This encapsulates all five dynamical systems (setpoint generator, position controller, plant, multi-sensor array, and observer) and their states, making it easy to reuse and experiment with different configurations. This pattern is particularly useful for integration with ROS2, as we'll see in later tutorials.


In [7]:
def initialize_crazyflie_system(
    # Initial states
    xk_init: np.ndarray = None,
    xhat_init: np.ndarray = None,
    P_init: np.ndarray = None,
    # System parameters
    dt: float = 0.01,  # 100 Hz
    Kp: float = 0.8,
    # Noise parameters
    Q: np.ndarray = None,
    R_mocap: np.ndarray = None,
    R_baro: np.ndarray = None,
    R_imu: np.ndarray = None,
    # Setpoint parameters
    initial_hover: np.ndarray = None,
    transitions: list = None
):
    """Initialize complete Crazyflie multi-sensor fusion system."""
    # Default initialization
    if xk_init is None:
        xk_init = np.array([[0.0], [0.0], [1.0], [0.0], [0.0], [0.0]])
    if xhat_init is None:
        xhat_init = xk_init.copy()
    if P_init is None:
        P_init = np.diag([0.1, 0.1, 0.1, 1.0, 1.0, 1.0])
    if Q is None:
        Q = np.diag([0.001, 0.001, 0.001, 0.1, 0.1, 0.1])
    if R_mocap is None:
        R_mocap = np.diag([0.005, 0.005, 0.005])
    if R_baro is None:
        R_baro = np.array([[0.1]])
    if R_imu is None:
        R_imu = np.diag([0.1, 0.1, 0.1])
    if initial_hover is None:
        initial_hover = np.array([[0.0], [0.0], [1.0]])
    
    # Initialize states
    xk = xk_init.copy()
    xhat_P = (xhat_init.copy(), P_init.copy())
    
    # Initialize setpoint generator
    setpoint_gen = initialize_hovering_setpoint(initial_hover, transitions)
    
    def callback(tk: float) -> dict:
        nonlocal xk, xhat_P
        
        # 1. Get setpoint
        rk = setpoint_gen(tk)
        
        # 2. Extract position estimate
        xhat = KF_module.h(xhat_P)
        phat = xhat[:3]
        
        # 3. Generate velocity command
        uk = position_controller(phat, rk, Kp=Kp)
        
        # 4. Plant step
        xk, _ = crazyflie_plant.step(
            param_dict={"xk": xk, "uk": uk, "dt": dt},
            return_state=True
        )
        
        # 5. Generate multi-sensor measurements
        yk_combined, R_combined = generate_multisensor_measurement(
            xk, R_mocap, R_baro, R_imu
        )
        
        # 6. Observer step
        Fk = compute_F_crazyflie(dt)
        Hk = compute_H_multisensor()
        
        xhat_P, xhat = crazyflie_observer.step(
            param_dict={
                "xhat_P": xhat_P,
                "yk": yk_combined,
                "f": crazyflie_f,
                "f_params": {"xk": xhat, "uk": uk, "dt": dt},
                "h": h_multisensor,
                "h_params": {"xk": xhat},
                "Fk": Fk,
                "Qk": Q,
                "Hk": Hk,
                "Rk": R_combined
            },
            return_state=True
        )
        
        # Return all signals
        return {
            "time": tk,
            "reference": rk.flatten(),
            "command": uk.flatten(),
            "true_state": xk.flatten(),
            "measurement_mocap": yk_combined[:3].flatten(),
            "measurement_baro": yk_combined[3].item(),
            "measurement_imu": yk_combined[4:].flatten(),
            "estimate": xhat.flatten(),
            "estimation_error": (xk - xhat).flatten()
        }
    
    return callback

Now we can run the same simulation with a much cleaner interface. All system parameters can be configured at initialization, and the simulation loop becomes trivial:


In [ ]:
# Initialize the complete Crazyflie navigation system
np.random.seed(42)
crazyflie_system = initialize_crazyflie_system(
    # Initial states
    xk_init=np.array([[0.0], [0.0], [1.0], [0.0], [0.0], [0.0]]),
    xhat_init=np.array([[0.0], [0.0], [1.0], [0.0], [0.0], [0.0]]),
    P_init=np.diag([0.1, 0.1, 0.1, 1.0, 1.0, 1.0]),
    # System parameters
    dt=0.01,
    Kp=0.8,
    # Kalman filter parameters
    Q=np.diag([0.001, 0.001, 0.001, 0.1, 0.1, 0.1]),
    R_mocap=np.diag([0.005, 0.005, 0.005]),
    R_baro=np.array([[0.1]]),
    R_imu=np.diag([0.1, 0.1, 0.1]),
    # Setpoint parameters
    initial_hover=np.array([[0.0], [0.0], [1.0]]),
    transitions=[
        (20.0, [1.0, 1.0, 1.5]),
        (40.0, [0.0, 0.0, 1.0]),
    ],
)

# Simulation time
T_sim = 60.0
dt = 0.01
time_steps = np.arange(0, T_sim, dt)

# Storage for results
results = {
    "time": [],
    "reference": [],
    "command": [],
    "true_state": [],
    "measurement_mocap": [],
    "measurement_baro": [],
    "measurement_imu": [],
    "estimate": [],
    "estimation_error": [],
}

# Run simulation
for tk in time_steps:
    output = crazyflie_system(tk)

    results["time"].append(output["time"])
    results["reference"].append(output["reference"])
    results["command"].append(output["command"])
    results["true_state"].append(output["true_state"])
    results["measurement_mocap"].append(output["measurement_mocap"])
    results["measurement_baro"].append(output["measurement_baro"])
    results["measurement_imu"].append(output["measurement_imu"])
    results["estimate"].append(output["estimate"])
    results["estimation_error"].append(output["estimation_error"])

# Convert to numpy arrays for plotting
for key in ["reference", "command", "true_state", "measurement_mocap", 
            "measurement_imu", "estimate", "estimation_error"]:
    results[key] = np.array(results[key])
results["measurement_baro"] = np.array(results["measurement_baro"])


In [ ]:
# Visualize results from callback
fig = plt.figure(figsize=(15, 10))

# Plot 1: 3D Trajectory
ax = fig.add_subplot(2, 3, 1, projection='3d')
ax.plot(
    results["true_state"][:, 0],
    results["true_state"][:, 1],
    results["true_state"][:, 2],
    'b-',
    label='True',
    linewidth=2,
    alpha=0.7
)
ax.plot(
    results["estimate"][:, 0],
    results["estimate"][:, 1],
    results["estimate"][:, 2],
    'r--',
    label='Estimate',
    linewidth=2,
    alpha=0.7
)
ax.scatter(
    results["reference"][:, 0],
    results["reference"][:, 1],
    results["reference"][:, 2],
    c='green',
    s=100,
    marker='*',
    label='Setpoints'
)
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title('3D Trajectory (Callback Interface)', fontweight='bold')
ax.legend()

# Plot 2: XY Trajectory (top view)
ax = fig.add_subplot(2, 3, 2)
ax.plot(
    results["true_state"][:, 0],
    results["true_state"][:, 1],
    'b-',
    label='True',
    linewidth=2,
    alpha=0.7
)
ax.plot(
    results["estimate"][:, 0],
    results["estimate"][:, 1],
    'r--',
    label='Estimate',
    linewidth=2,
    alpha=0.7
)
ax.scatter(
    results["reference"][:, 0],
    results["reference"][:, 1],
    c='green',
    s=100,
    marker='*',
    label='Setpoints',
    zorder=5
)
ax.set_xlabel('X Position (m)', fontsize=11)
ax.set_ylabel('Y Position (m)', fontsize=11)
ax.set_title('XY Trajectory (Top View)', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')

# Plot 3: Z Position vs Time (with all sensors)
ax = fig.add_subplot(2, 3, 3)
ax.plot(
    results["time"],
    results["true_state"][:, 2],
    'b-',
    label='True Z',
    linewidth=2,
    alpha=0.7
)
ax.plot(
    results["time"],
    results["estimate"][:, 2],
    'r--',
    label='Estimated Z',
    linewidth=2,
    alpha=0.7
)
ax.scatter(
    results["time"][::100],
    results["measurement_mocap"][::100, 2],
    c='green',
    s=5,
    alpha=0.3,
    label='Mocap Z'
)
ax.scatter(
    results["time"][::50],
    results["measurement_baro"][::50],
    c='purple',
    s=5,
    alpha=0.3,
    label='Baro Z'
)
ax.set_ylabel('Z Position (m)', fontsize=11)
ax.set_title('Altitude: Multi-Sensor Fusion', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: X Position vs Time
ax = fig.add_subplot(2, 3, 4)
ax.plot(
    results["time"],
    results["true_state"][:, 0],
    'b-',
    label='True X',
    alpha=0.7
)
ax.plot(
    results["time"],
    results["estimate"][:, 0],
    'r--',
    label='Estimated X',
    alpha=0.7
)
ax.scatter(
    results["time"][::100],
    results["measurement_mocap"][::100, 0],
    c='gray',
    s=5,
    alpha=0.3,
    label='Mocap'
)
ax.set_ylabel('X Position (m)', fontsize=11)
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_title('X Position Over Time', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 5: Velocity Commands
ax = fig.add_subplot(2, 3, 5)
ax.plot(results["time"], results["command"][:, 0], label='Vx cmd', alpha=0.7)
ax.plot(results["time"], results["command"][:, 1], label='Vy cmd', alpha=0.7)
ax.plot(results["time"], results["command"][:, 2], label='Vz cmd', alpha=0.7)
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('Velocity Command (m/s)', fontsize=11)
ax.set_title('Control Commands', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 6: Position Estimation Error
ax = fig.add_subplot(2, 3, 6)
position_error = np.sqrt(
    results["estimation_error"][:, 0]**2 + 
    results["estimation_error"][:, 1]**2 + 
    results["estimation_error"][:, 2]**2
)
ax.plot(
    results["time"],
    position_error * 1000,
    'm-',
    linewidth=1.5
)  # Convert to mm
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('Position Error (mm)', fontsize=11)
ax.set_title('3D Position Estimation Error', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean position error: {np.mean(position_error)*1000:.2f} mm")
print(f"Max position error: {np.max(position_error)*1000:.2f} mm")


## Experimentation

Now that we have a working system, try experimenting with different parameters:

**Sensor characteristics**:
- Vary mocap noise to simulate poor lighting conditions
- Increase barometer noise to simulate temperature-induced drift
- Increase IMU noise to simulate cheap sensors
- Observe how the KF automatically reweights sensors

**Controller tuning**:
- Adjust `Kp` to change position tracking aggressiveness
- Try different hover patterns (figure-eight, circle, random)
- Add velocity limits to simulate real quadrotor constraints

**Observer comparison**:
- Run simulation with KF disabled (use raw mocap only)
- Compare estimation error with single-sensor vs multi-sensor
- Visualize which sensor dominates in different flight regimes

**Challenge**: Design a trajectory where fusion of all three sensors significantly outperforms any single sensor.


[← Control Systems as Dynamical Systems](../../../getting_started/theory_to_python/control_systems_as_dynamical_systems.rst)
